In [21]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import re


In [14]:
# function that finds a word in a sentence, only a word and not a subword
def find_whole_word(w):
    return re.compile(r'\b({0})\b'.format(w), flags=re.IGNORECASE).search

# match for every candidate the words and count simple
# check in the messages how many matches with the words from the list
def find_words_candidate_messages(messages, words_list, filename_to_save):

  # define structures to store results
  candidates_word_counts = pd.DataFrame(columns = ["candidate_id","word_counts","total"])
  messages_with_words = pd.DataFrame(columns = ["candidate_id","message","word"])

  # get list of candidates ids
  candidates_list = np.unique(messages['candidate_id'])

  # for all candidates
  for candidate in candidates_list:
    sum_words_counts = 0
    messages_text = messages.loc[messages['candidate_id'] == candidate]['text']

    # for all messages of one candidate
    for message in messages_text:

      # match for all words
      for word in words_list:
        result = find_whole_word(word)(message)
        if(result != None):
          sum_words_counts = sum_words_counts + 1
          messages_with_words = messages_with_words.append({'candidate_id': candidate,
                                                            'message': message,
                                                            'word': word}, ignore_index=True)
        
    candidates_word_counts = candidates_word_counts.append({'candidate_id': candidate,
                                                            'word_counts': sum_words_counts,
                                                            'total': len(messages_text)}, ignore_index=True)

  # normalize the counts with the total messages from a candidate
  candidates_word_counts['words_percentage'] = candidates_word_counts['word_counts'] / candidates_word_counts['total']

  # save to csv
  candidates_word_counts.to_csv("./resulting_files/" + filename_to_save + '_percentages.csv', index=False, sep=',')
  messages_with_words.to_csv("./resulting_files/" + filename_to_save + '_messages.csv', index=False, sep=',')

# convert the minority words to minority category
def convert_minority_words_to_category(minority_words_messages, categories_list_file, minority_words_percentages):
  # join with the original word list table to get the category
  result = pd.merge(minority_words_messages, categories_list_file, how="inner", on=["word"])
  # compute frequencies of minority
  result = pd.DataFrame(result.groupby(["candidate_id", "minority"]).size().unstack(fill_value=0).stack().reset_index(name='count'))
  # compute total frequencies of minority per candidate
  compute_total_words_minorities = result.groupby(by='candidate_id').sum().reset_index()
  result = pd.merge(result, compute_total_words_minorities, how="inner", on=["candidate_id"])
  # merge also the total messages per candidate
  num_total_messages_candidate = minority_words_percentages[["candidate_id","total"]]
  result = pd.merge(result, num_total_messages_candidate, how="inner", on=["candidate_id"])
  # rename columns for better reading
  result = result.rename(columns={'count_x': 'num_words_minority', 'count_y': 'total_words_minority', 'total': 'total_messages'})

  # normalize with the total messages from one candidate
  result["percentage_minority_inside_all_candidate_messages"] = result["num_words_minority"] / result["total_messages"]
  
  # save results to csv
  result.to_csv('./resulting_files/data_radar_plots.csv', index=False, sep=',')

             word    minority insult
0          cigana     ciganos     no
1         ciganas     ciganos     no
2          cigano     ciganos     no
3         ciganos     ciganos     no
4            lela     ciganos    yes
..            ...         ...    ...
159   intersexual        LGBT     no
160  intersexuais        LGBT     no
161   não binário        LGBT     no
162     refugiado  refugiados     no
163    refugiados  refugiados     no

[164 rows x 3 columns]


In [42]:
# get words list
url = 'https://raw.githubusercontent.com/paulafortuna/Portuguese-minority-terms/main/final_list.csv'
final_list_words = pd.read_csv(url)
minority_words = final_list_words.loc[final_list_words['minority'] != "no"]["word"]
insult_words = final_list_words.loc[final_list_words['insult'] == "yes"]["word"]
minority_insult_words = pd.Series(list(set(minority_words) & set(insult_words)))

Usage example

In [43]:
# get messages
messages_candidates_replies = pd.read_csv('./resulting_files/candidates_replies.csv', sep=',')

In [44]:
# call functions for computing some metrics
find_words_candidate_messages(messages_candidates_replies, minority_words, 'minority_words')
find_words_candidate_messages(messages_candidates_replies, insult_words, 'insult_words')
find_words_candidate_messages(messages_candidates_replies, minority_insult_words, 'minority_insult_words')

In [45]:
# read computed metrics
minority_words_percentages = pd.read_csv('./resulting_files/minority_words_percentages.csv', sep=',')
insult_words_percentages = pd.read_csv('./resulting_files/insult_words_percentages.csv', sep=',')
minority_insult_words_percentages = pd.read_csv('./resulting_files/minority_insult_words_percentages.csv', sep=',')

minority_words_messages = pd.read_csv('./resulting_files/minority_words_messages.csv', sep=',')

In [46]:
convert_minority_words_to_category(minority_words_messages,final_list_words,minority_words_percentages)
